<a href="https://colab.research.google.com/github/Kumarvels/GenAIProjects/blob/main/Cybersecurity_AutoPentest_Agents1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What: Project Overview

Cyber-AutoAgent: Agents Penetration Testing using multiple pentesting tools and LLMs, and the addition of a detailed dashboard.

Why:  

It explains the need for comprehensive testing, industry-standard reporting, and visual analytics through a dashboard to enhance decision-making and compliance.

How: Implementation Steps

The code implements the entire system, including LLM configuration, Mem0, Strands Agents SDK, pentesting tools, the CyberAutoAgent class, progress tracking, and industry-standard report automation.

The dashboard is created using Streamlit and Plotly, featuring multiple interactive visualizations:

Assessment Progress: Shows the current progress as a percentage.

Vulnerability Distribution: A bar chart displaying vulnerabilities by risk level.

Tool Usage Summary: A pie chart summarizing the tools used.

Risk Trends Over Time: A line chart showing historical risk trends.

Recommendations: Displays the industry-standard report content.

Outcome: Expected Results
The outcome section details the expected results, including comprehensive security assessments, standardized reports, a detailed dashboard, and continuous improvement.

This single-cell notebook provides a complete, self-contained solution for DAST with a beautiful, interactive dashboard, ready for execution in Google Colab. The dashboard enhances the presentation of results, making it easier for stakeholders to understand and act on the security assessment findings.



**Cybersecurity Penetration Testing Automation - Agents based Orchestration**

In [ ]:
import os
import sys
import time
import subprocess
import pandas as pd
import google.generativeai as genai
from mem0 import Memory
from typing import Dict, Any
!pip install streamlit plotly pandas genai

# Conditional import for nmap, ipywidgets, and plotly/streamlit for Colab/Dashboard
try:
    import nmap
except ImportError:
    print("python-nmap not found. Please install it: pip install python-nmap")
    nmap = None

try:
    import ipywidgets as widgets
    from IPython.display import display
except ImportError:
    print("ipywidgets not found. Progress bar will be printed to console.")
    widgets = None
    display = None


## Section: LLM Manager
# What: The LLMManager class is responsible for managing different Large Language Models (LLMs).
# Why: It provides a centralized way to add, configure, and retrieve various LLMs (like Gemini, OpenAI, etc.). This makes the code more modular and flexible, allowing easy switching or adding new LLMs without modifying core logic.
# How: It uses a dictionary to store LLM configurations, each with a name (e.g., "gemini") and a dictionary containing the LLM client and a configuration function.
# Outcomes: A streamlined way to handle multiple LLM integrations, improving maintainability and extensibility.
class LLMManager:
    def __init__(self):
        self.llms = {}

    def add_llm(self, name: str, config: Dict[str, Any]):
        self.llms[name] = config

    def get_llm(self, name: str):
        if name not in self.llms:
            raise ValueError(f"LLM {name} not configured.")
        return self.llms[name]

llm_manager = LLMManager()
# Assuming GOOGLE_API_KEY is set in environment variables or Colab secrets
llm_manager.add_llm("gemini", {"client": genai.GenerativeModel("gemini-pro"), "configure": lambda: genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))})

try:
    from azure.ai.textanalytics import TextAnalyticsClient
    from azure.core.credentials import AzureKeyCredential
    llm_manager.add_llm("azure_ai", {"client": TextAnalyticsClient(endpoint="https://your-azure-endpoint.cognitiveservices.azure.com/", credential=AzureKeyCredential(os.environ["AZURE_AI_KEY"])), "configure": lambda: None})
except ImportError:
    print("Azure Text Analytics modules not found. Skipping Azure AI LLM configuration.")
except KeyError:
    print("AZURE_AI_KEY environment variable not set. Skipping Azure AI LLM configuration.")


## Section: Mem0 Configuration
# What: This segment initializes the Mem0 memory client, which is used for persistent storage and retrieval of assessment data.
# Why: Memory is crucial for an agent to learn from past interactions, store assessment results, and provide context for future actions or reports. It prevents redundant work and enables more intelligent decision-making.
# How: It attempts to initialize `mem0.Memory` by passing the `gemini-pro` LLM client. It includes error handling for cases where the LLM configuration or Mem0 initialization fails.
# Outcomes: The `memory` object is available for the CyberAgent to store and retrieve DAST findings, enabling statefulness and context-awareness.
try:
    gemini_client_for_memory = llm_manager.get_llm("gemini")["client"]
    try:
        memory = Memory(llm=gemini_client_for_memory)
    except TypeError:
        print("Attempted to initialize Mem0 with 'llm' parameter, failed. Trying 'client'.")
        try:
            memory = Memory(client=gemini_client_for_memory)
        except TypeError:
            print("Error: Mem0 initialization failed with both 'llm' and 'client' arguments. Check Mem0 documentation.")
            memory = None
except ValueError as e:
    print(f"Error configuring Mem0: {e}. Make sure Gemini API key is set.")
    memory = None
except Exception as e:
    print(f"Error initializing Mem0: {e}. Check Mem0 documentation for correct initialization.")
    memory = None


## Section: Tool Assessment Functions
# What: These are individual Python functions, each representing a specific cybersecurity assessment tool (e.g., ZAP, SQLMap, Nikto, Nmap).
# Why: They encapsulate the logic for executing external command-line tools or interacting with APIs of security assessment platforms. This modularity allows the agent to call diverse tools based on the assessment needs.
# How: Each function typically uses `subprocess.run` to execute a command-line tool with specific arguments or contains placeholder logic for API interactions. They capture the output and return it. Error handling is included for missing tools or execution failures.
# Outcomes: The agent can leverage a variety of DAST tools programmatically, widening its assessment capabilities. Each function provides a standardized interface for tool execution.
def run_zap_assessment(url):
    # What: Placeholder for ZAP (OWASP ZAP) assessment.
    # Why: ZAP is a popular open-source DAST tool for finding vulnerabilities in web applications.
    # How: In a real scenario, this would involve interacting with ZAP's API to initiate spidering, active scans, and retrieve alerts. Currently, it simulates results.
    # Outcomes: (Simulated) DAST findings like SQL Injection and Cross-Site Scripting.
    try:
        print(f"Running ZAP assessment on {url}")
        alerts = [{'alert': 'SQL Injection', 'risk': 'High', 'url': url},
                  {'alert': 'Cross-Site Scripting', 'risk': 'Medium', 'url': url}]
        return alerts
    except Exception as e:
        print(f"Error running ZAP assessment: {e}")
        return f"ZAP assessment failed: {e}"

def run_burp_assessment(url):
    # What: Placeholder for Burp Suite assessment.
    # Why: Burp Suite is a leading integrated platform for web application security testing.
    # How: In a real scenario, this would involve Burp's API for automated scans or integrating with its proxy for manual testing data.
    # Outcomes: (Simulated) An indication that a Burp Suite assessment was performed.
    print(f"Running Burp Suite assessment on {url}")
    return "Burp Suite assessment simulated."

def run_sqlmap_assessment(url):
    # What: Executes SQLMap for SQL Injection vulnerability detection.
    # Why: SQLMap is an open-source tool for automating the process of detecting and exploiting SQL injection flaws.
    # How: Uses `subprocess.run` to call the `sqlmap` command with the target URL.
    # Outcomes: Output from SQLMap detailing potential SQL injection vulnerabilities.
    try:
        result = subprocess.run(['sqlmap', '-u', url, '--batch'], capture_output=True, text=True, check=True)
        return result.stdout
    except FileNotFoundError:
        return "sqlmap not found. Please install it and ensure it's in your PATH."
    except subprocess.CalledProcessError as e:
        return f"sqlmap assessment failed: {e.stderr}"
    except Exception as e:
        return str(e)

def run_nikto_assessment(url):
    # What: Executes Nikto for web server vulnerability scanning.
    # Why: Nikto is a web server scanner that tests web servers for dangerous files/CGIs, outdated server software, and other problems.
    # How: Uses `subprocess.run` to call the `nikto` command with the target URL.
    # Outcomes: Output from Nikto detailing web server configuration issues and potential vulnerabilities.
    try:
        result = subprocess.run(['nikto', '-h', url], capture_output=True, text=True, check=True)
        return result.stdout
    except FileNotFoundError:
        return "nikto not found. Please install it and ensure it's in your PATH."
    except subprocess.CalledProcessError as e:
        return f"nikto assessment failed: {e.stderr}"
    except Exception as e:
        return str(e)

def run_nessus_assessment(url):
    # What: Placeholder for Nessus vulnerability scanning.
    # Why: Nessus is a widely used vulnerability scanner, often for network and system-level vulnerabilities, but can include web applications.
    # How: In a real implementation, this would involve Nessus API calls to initiate scans and retrieve reports.
    # Outcomes: (Simulated) An indication that a Nessus assessment was performed.
    try:
        print(f"Running Nessus assessment on {url}")
        return "Nessus assessment simulated."
    except FileNotFoundError:
        return "Nessus not found. Please install it."
    except Exception as e:
        return str(e)

def run_nmap_assessment(url):
    # What: Executes Nmap for network discovery and security auditing.
    # Why: Nmap is an essential tool for network reconnaissance, identifying open ports, services, and operating systems.
    # How: Uses the `python-nmap` library to perform a service version detection scan (`-sV`) on the target URL's host.
    # Outcomes: Network and service information (e.g., open ports, running services) in CSV format.
    try:
        if nmap:
            nm = nmap.PortScanner()
            nm.scan(url, arguments='-sV')
            return nm.csv()
        else:
            return "nmap (python-nmap) module not available."
    except FileNotFoundError:
        return "nmap executable not found. Please install it and ensure it's in your PATH."
    except Exception as e:
        print(f"Error running Nmap assessment: {e}")
        return f"nmap assessment failed: {e}"

def run_metastalker_assessment(url):
    # What: Executes Metastalker for metadata extraction from publicly available documents.
    # Why: Metadata can sometimes reveal sensitive information about an organization or its infrastructure.
    # How: Uses `subprocess.run` to call the `metastalker` command with the target URL.
    # Outcomes: Extracted metadata, which could include author names, software versions, or creation dates.
    try:
        result = subprocess.run(['metastalker', '-u', url], capture_output=True, text=True, check=True)
        return result.stdout
    except FileNotFoundError:
        return "metastalker not found. Please install it and ensure it's in your PATH."
    except subprocess.CalledProcessError as e:
        return f"metastalker assessment failed: {e.stderr}"
    except Exception as e:
        return str(e)

def run_wpscan_assessment(url):
    # What: Executes WPScan for WordPress vulnerability scanning.
    # Why: WPScan is a specialized tool for identifying vulnerabilities in WordPress installations, including themes, plugins, and core.
    # How: Uses `subprocess.run` to call the `wpscan` command, formatted for JSON output.
    # Outcomes: JSON output containing detected vulnerabilities, theme/plugin issues, and user enumeration results for WordPress sites.
    try:
        result = subprocess.run(['wpscan', '--url', url, '--batch', '--format', 'json'], capture_output=True, text=True, check=True)
        import json
        try:
            return json.loads(result.stdout)
        except json.JSONDecodeError:
            return result.stdout
    except FileNotFoundError:
        return "wpscan not found. Please install it and ensure it's in your PATH."
    except subprocess.CalledProcessError as e:
        return f"wpscan assessment failed: {e.stderr}"
    except Exception as e:
        return str(e)

def run_dirb_assessment(url):
    # What: Executes Dirb for web content scanning (bruteforcing directories and files).
    # Why: Dirb is used to find hidden web objects (directories, files, etc.) that might not be linked from the main site.
    # How: Uses `subprocess.run` to call the `dirb` command, outputting results to a temporary file.
    # Outcomes: A list of discovered directories and files, which could indicate misconfigurations or sensitive data.
    try:
        result = subprocess.run(['dirb', url, '-o', '/tmp/dirb_output.txt'], capture_output=True, text=True)
        with open('/tmp/dirb_output.txt', 'r') as f:
            output = f.read()
        return output
    except FileNotFoundError:
        return "dirb not found. Please install it and ensure it's in your PATH."
    except Exception as e:
        return str(e)

def run_arachni_assessment(url):
    # What: Executes Arachni for web application security scanning.
    # Why: Arachni is a feature-packed Ruby framework designed to help penetration testers and administrators evaluate the security of web applications.
    # How: Uses `subprocess.run` to call the `arachni` command, formatted for JSON output.
    # Outcomes: JSON output detailing various web application vulnerabilities detected by Arachni.
    try:
        result = subprocess.run(['arachni', '--url', url, '--output-format', 'json'], capture_output=True, text=True, check=True)
        import json
        try:
            return json.loads(result.stdout)
        except json.JSONDecodeError:
            return result.stdout
    except FileNotFoundError:
        return "arachni not found. Please install it and ensure it's in your PATH."
    except subprocess.CalledProcessError as e:
        return f"arachni assessment failed: {e.stderr}"
    except Exception as e:
        return str(e)


## Section: ToolManager Class
# What: The ToolManager class centralizes the management and execution of various cybersecurity assessment tools.
# Why: It provides a single interface to access and run all defined assessment tools. This abstracts the complexity of calling different tools and ensures consistency. It also includes checks for tool availability.
# How: It stores a dictionary of tool names mapped to their respective functions. The `run_tool` method verifies if a tool is present and executable before attempting to run it, then calls the corresponding function.
# Outcomes: A robust and extensible system for orchestrating DAST tool execution, with built-in checks for tool readiness.
class ToolManager:
    def __init__(self):
        self.tools = {
            "zap": run_zap_assessment,
            "burp": run_burp_assessment,
            "sqlmap": run_sqlmap_assessment,
            "nikto": run_nikto_assessment,
            "nessus": run_nessus_assessment,
            "nmap": run_nmap_assessment,
            "metastalker": run_metastalker_assessment,
            "wpscan": run_wpscan_assessment,
            "dirb": run_dirb_assessment,
            "arachni": run_arachni_assessment
        }

    def add_tool(self, name, function):
        self.tools[name] = function

    def run_tool(self, name, url):
        if name in self.tools:
            try:
                # Basic check for some common tools
                if name in ["sqlmap", "nikto", "metastalker", "wpscan", "dirb", "arachni"]:
                    subprocess.run([name, '--version'], capture_output=True, text=True, check=True)
                elif name == "nmap":
                    subprocess.run(['nmap', '--version'], capture_output=True, text=True, check=True)
                return self.tools[name](url)
            except (FileNotFoundError, subprocess.CalledProcessError):
                print(f"Warning: Tool '{name}' not found or not executable. Skipping assessment with this tool.")
                return f"Tool '{name}' not found or not executable."
            except Exception as e:
                print(f"Error running tool '{name}': {e}")
                return str(e)
        return "Tool not found."

tool_manager = ToolManager()

## Section: GeminiCyberAgent Class
# What: The core class of the cybersecurity agent, orchestrating LLM interactions, tool execution, and report generation.
# Why: This class brings together all components (LLM, ToolManager, Memory) to perform comprehensive DAST assessments, analyze results, and generate human-readable reports.
# How: It initializes with an LLM client, a ToolManager instance, and a memory client. It contains methods for performing DAST, generating reports, and providing analytics.
# Outcomes: A functional and intelligent agent capable of autonomous security assessments and reporting.
class GeminiCyberAgent:
    def __init__(self, llm_client, tool_manager, memory=None):
        # What: Constructor for the GeminiCyberAgent.
        # Why: Initializes the agent with necessary components for its operations.
        # How: Takes an LLM client, a tool manager, and an optional memory instance as arguments and assigns them to instance variables. Also initializes progress tracking.
        # Outcomes: A ready-to-use CyberAgent instance.
        self.llm_client = llm_client
        self.tool_manager = tool_manager
        self.memory = memory
        self.progress = 0
        self.total_tasks = 0

    def perform_dast(self, url):
        # What: Performs a Dynamic Application Security Test (DAST) on a given URL.
        # Why: This is the primary function for initiating a security assessment. It intelligently selects the best tool using the LLM and then executes it.
        # How: It constructs a prompt for the LLM to choose the most appropriate tool from the available list, considering past assessments. It then runs the chosen tool and stores the results in memory.
        # Outcomes: DAST findings from the selected tool, stored in memory for later reporting and analysis.
        if not self.llm_client:
            print("LLM client not initialized.")
            return "Assessment skipped."

        self.total_tasks += 1

        past_assessments_context = ""
        if self.memory:
            past_assessments = self.memory.search("DAST for " + url, user_id="guest")
            if past_assessments:
                past_assessments_context = "Previous DAST findings: " + str(past_assessments)

        available_tools = list(self.tool_manager.tools.keys())
        prompt = f"""You are an expert in web application security testing. Your task is to select the best tool for performing a dynamic application security test (DAST) on the following URL: {url}.
Available tools: {', '.join(available_tools)}.
{past_assessments_context}
Based on the URL and any previous findings, which tool would be the most effective? Respond with only the name of the chosen tool from the available tools list (e.g., "zap", "sqlmap", "nmap").
"""
        try:
            safety_settings = [
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
            ]
            response = self.llm_client.generate_content(prompt, safety_settings=safety_settings)
            tool_choice = response.text.strip().lower()

            if tool_choice not in available_tools:
                print(f"LLM chose an invalid tool: {tool_choice}. Defaulting to 'zap'.")
                tool_choice = "zap"

            print(f"LLM chose tool: {tool_choice}")

            results = self.tool_manager.run_tool(tool_choice, url)

            if self.memory:
                self.memory.add(f"DAST results for {url} using {tool_choice}: {str(results)}")

            self.update_progress()
            return results

        except Exception as e:
            print(f"An error occurred during LLM interaction or tool execution: {e}")
            return f"Assessment failed: {e}"


    def generate_report(self, url):
        # What: Generates a general security assessment report.
        # Why: To summarize the findings from the DAST assessment in a human-readable format.
        # How: Retrieves stored DAST assessments from memory, then uses the LLM to synthesize this information into a cohesive report, including an executive summary, findings, and recommendations.
        # Outcomes: A text-based security report summarizing the assessment results.
        if self.memory:
            assessments = self.memory.search("DAST for " + url, user_id="guest")
            print(f"Generating report based on: {assessments}")
            if self.llm_client:
                report_prompt = f"Generate a security assessment report for {url} based on the following findings: {str(assessments)}. Include an executive summary, detailed findings by risk level, and recommendations."
                try:
                    report_response = self.llm_client.generate_content(report_prompt)
                    return report_response.text
                except Exception as e:
                    print(f"Error generating report with LLM: {e}")
                    return "Report generation failed."
            else:
                return "LLM client not available for report generation."
        else:
            print("Memory not available, cannot generate report.")
            return "Report generation skipped. Memory not available."

    def generate_analytics(self, url):
        # What: Generates analytical insights from the assessment data.
        # Why: To provide a structured, quantifiable overview of vulnerabilities, useful for dashboards and trend analysis.
        # How: Fetches assessment data from memory, filters for structured (dictionary) findings, and converts them into a Pandas DataFrame. It specifically looks for a 'risk' column to categorize vulnerabilities.
        # Outcomes: A Pandas DataFrame containing structured assessment data, suitable for further analysis and visualization (e.g., vulnerability distribution charts).
        if self.memory:
            assessments = self.memory.search("DAST for " + url, user_id="guest")
            try:
                valid_assessments = [a for a in assessments if isinstance(a, dict)]
                if valid_assessments:
                    df = pd.DataFrame(valid_assessments)
                    if 'risk' in df.columns:
                        df['risk'] = df['risk'].str.capitalize()
                        return df
                    else:
                        print("Warning: 'risk' column not found in assessment data. Cannot generate vulnerability distribution.")
                        return pd.DataFrame(valid_assessments)
                else:
                    print("No valid assessment data (dictionary format) to generate analytics.")
                    return pd.DataFrame()

            except Exception as e:
                print(f"Error creating DataFrame from assessments: {e}")
                return pd.DataFrame()
        else:
            print("Memory not available, cannot generate analytics.")
            return pd.DataFrame()


    def generate_industry_standard_report(self, url):
        # What: Generates a comprehensive, industry-standard security assessment report.
        # Why: Provides a more formal and detailed report suitable for various stakeholders, aligning with typical cybersecurity report structures.
        # How: Retrieves structured assessment data from memory. It then prompts the LLM to generate content for predefined sections (Executive Summary, Methodology, Findings by Risk, Recommendations). It includes a fallback template if the LLM's response isn't perfectly structured.
        # Outcomes: A detailed Markdown-formatted report, organized into standard sections, ideal for official documentation.
        if not self.llm_client:
            return "Industry standard report generation skipped. LLM client not initialized."

        assessments = self.memory.search("DAST for " + url, user_id="guest") if self.memory else []
        valid_assessments = [a for a in assessments if isinstance(a, dict)]

        if not valid_assessments and self.memory:
            print("No valid assessment data (dictionary format) found in memory for industry report.")
            return f"# Security Assessment Report for {url}\n\n## Executive Summary\nNo structured vulnerability data found in memory.\n\n## Findings\nNo structured findings available.\n\n## Recommendations\nNo specific recommendations based on structured data.\n\nPrepared by: CyberAutoAgent\nDate: {time.strftime('%Y-%m-%d')}"
        elif not self.memory:
            return "Industry standard report generation skipped. Memory not available."

        try:
            report_content_prompt = f"""Generate content for a security assessment report based on the following findings for {url}: {str(valid_assessments)}.
            Provide the following sections:
            1.  Executive Summary: A brief overview of the assessment and key findings.
            2.  Methodology: Mention the types of tools used (e.g., DAST tools).
            3.  Findings (High-Risk): Detail high-risk vulnerabilities.
            4.  Findings (Medium-Risk): Detail medium-risk vulnerabilities.
            5.  Findings (Low-Risk): Detail low-risk vulnerabilities.
            6.  Recommendations: Actionable steps to mitigate identified vulnerabilities.
            Format the response clearly with section headers."""

            safety_settings = [
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
            ]
            report_response = self.llm_client.generate_content(report_content_prompt, safety_settings=safety_settings)
            report_content = report_response.text

            sections = report_content.split("## ")
            report_dict = {}
            for section in sections:
                if section.strip():
                    lines = section.split('\n', 1)
                    if len(lines) > 1:
                        header = lines[0].strip()
                        content = lines[1].strip()
                        report_dict[header] = content

            if not report_dict:
                print("Warning: LLM report generation did not produce expected sections. Falling back to template.")
                high_risk = [str(a) for a in valid_assessments if a.get('risk', '').lower() == 'high']
                medium_risk = [str(a) for a in valid_assessments if a.get('risk', '').lower() == 'medium']
                low_risk = [str(a) for a in valid_assessments if a.get('risk', '').lower() == 'low']

                recommendations_llm = ""
                if self.llm_client:
                    try:
                        rec_prompt = "Provide concise recommendations to mitigate the following web vulnerabilities: " + str(valid_assessments)
                        rec_response = self.llm_client.generate_content(rec_prompt)
                        recommendations_llm = rec_response.text
                    except Exception as e:
                        print(f"Error generating recommendations with LLM: {e}")
                        recommendations_llm = "Recommendations could not be generated."

                report_template = """
# Security Assessment Report for {url}

## Executive Summary
{executive_summary}

## Methodology
The assessment utilized dynamic analysis and tools such as {tools}.

## Findings
### High-Risk Vulnerabilities
{high_risk}

### Medium-Risk Vulnerabilities
{medium_risk}

### Low-Risk Vulnerabilities
{low_risk}

## Recommendations
{recommendations}

## Conclusion
The assessment identified {total_vulnerabilities} vulnerabilities.

Prepared by: CyberAutoAgent
Date: {date}
"""
                return report_template.format(
                    url=url,
                    executive_summary=report_dict.get('Executive Summary', 'No executive summary generated.'),
                    tools=', '.join(tool_manager.tools.keys()),
                    high_risk='\n'.join(high_risk) if high_risk else 'None identified.',
                    medium_risk='\n'.join(medium_risk) if medium_risk else 'None identified.',
                    low_risk='\n'.join(low_risk) if low_risk else 'None identified.',
                    recommendations=report_dict.get('Recommendations', recommendations_llm if recommendations_llm else 'No recommendations generated.'),
                    total_vulnerabilities=len(valid_assessments),
                    date=time.strftime("%Y-%m-%d")
                )
            return report_content

        except Exception as e:
            print(f"An error occurred during report generation with LLM: {e}")
            return f"Report generation failed: {e}"

    def update_progress(self):
        # What: Updates the progress of the assessment.
        # Why: Provides real-time feedback to the user on the agent's activity, especially useful in long-running tasks.
        # How: Increments a counter for completed tasks. If `ipywidgets` is available (as in Colab), it updates a graphical progress bar; otherwise, it prints to the console.
        # Outcomes: Improved user experience through visible progress tracking.
        self.progress += 1
        try:
            if 'ipywidgets' in sys.modules and 'progress_bar' in globals():
                progress_value = min(int((self.progress / self.total_tasks) * 100), 100)
                progress_bar.value = progress_value
                progress_bar.description = f'Progress: {self.progress}/{self.total_tasks} tasks completed'
        except Exception as e:
            print(f"Error updating progress bar: {e}")

        if 'ipywidgets' not in sys.modules or 'progress_bar' not in globals():
            if self.total_tasks > 0:
                print(f"Progress: {self.progress}/{self.total_tasks} tasks completed")


## Section: Agent Initialization and Example Usage
# What: This section initializes the `GeminiCyberAgent` and demonstrates how to use its core functionalities.
# Why: It serves as the entry point for running the security assessment process and showcasing the agent's capabilities from DAST to report generation.
# How: It retrieves the configured Gemini LLM client, instantiates the `GeminiCyberAgent` with the LLM, ToolManager, and Memory. Then, it calls `perform_dast`, `generate_report`, `generate_industry_standard_report`, and `generate_analytics` methods for a specified URL.
# Outcomes: Execution of a full DAST workflow, resulting in printed assessment results, various reports, and analytical data.
try:
    gemini_llm_client = llm_manager.get_llm("gemini")["client"]
    cyber_agent = GeminiCyberAgent(llm_client=gemini_llm_client, tool_manager=tool_manager, memory=memory)
except ValueError as e:
    print(f"Error initializing GeminiCyberAgent: {e}. Make sure Gemini API key is set and LLM is configured.")
    cyber_agent = None
except Exception as e:
    print(f"An unexpected error occurred during GeminiCyberAgent initialization: {e}")
    cyber_agent = None


url = "http://example.com"
if cyber_agent:
    cyber_agent.total_tasks = 1

    print(f"Starting DAST on {url}...")
    result = cyber_agent.perform_dast(url)
    print("\nDAST Result:")
    print(result)

    print("\nGenerating Report...")
    report = cyber_agent.generate_report(url)
    print("\nGenerated Report:")
    print(report)

    print("\nGenerating Industry Standard Report...")
    industry_report = cyber_agent.generate_industry_standard_report(url)
    print("\nGenerated Industry Standard Report:")
    print(industry_report)

    print("\nGenerating Analytics...")
    analytics = cyber_agent.generate_analytics(url)
    print("\nGenerated Analytics (DataFrame head):")
    if not analytics.empty:
        display(analytics.head())
    else:
        print("No analytics data to display.")


## Section: Progress Bar Display
# What: Initializes and displays an `ipywidgets` progress bar in environments like Google Colab.
# Why: To provide a visual indicator of the assessment progress, enhancing the user experience during potentially long-running operations.
# How: It checks for the availability of the `ipywidgets` module and, if present, creates an `IntProgress` widget and displays it. The `cyber_agent.update_progress()` method is responsible for updating this bar.
# Outcomes: A dynamic progress bar that visually tracks the completion of assessment tasks.
if 'ipywidgets' in sys.modules and 'progress_bar' not in globals():
    progress_bar = widgets.IntProgress(value=0, min=0, max=100, description='Progress:', bar_style='info', orientation='horizontal')
    display(progress_bar)
elif 'ipywidgets' not in sys.modules:
    print("ipywidgets not available, skipping progress bar display.")


## Section: Save Industry Standard Report
# What: Saves the generated industry-standard report to a Markdown file.
# Why: To persist the comprehensive report, making it available for offline viewing, sharing, or archiving.
# How: Checks if the `cyber_agent` is initialized and if an `industry_report` has been generated. If so, it creates a unique filename and writes the report content to a `.md` file.
# Outcomes: A physical file containing the detailed security assessment report, ready for external use.
if cyber_agent and industry_report:
    try:
        report_filename = f"security_report_{url.replace('http://', '').replace('https://', '').replace('/', '_')}_{time.strftime('%Y%m%d_%H%M%S')}.md"
        with open(report_filename, "w") as f:
            f.write(industry_report)
        print(f"Industry-standard report generated and saved as {report_filename}")
    except Exception as e:
        print(f"Error saving industry standard report: {e}")
else:
    print("Industry-standard report not saved because CyberAgent was not initialized or report was not generated.")



**Streamlit Dashboard (Currently Commented Out)**

What:

The commented-out code block is designed to create a web-based Streamlit dashboard for visualizing the security assessment data.

Why:

A dashboard provides a much more intuitive and interactive way to understand assessment results compared to raw text. It allows for quick insights into vulnerability distribution, tool usage, and potential risk trends.

How:

Streamlit Installation: Streamlit is a Python library for building interactive web applications. It needs to be installed separately (pip install streamlit plotly pandas).
Dashboard Components: The code uses Streamlit functions (st.title, st.header, st.plotly_chart, st.markdown) to lay out the dashboard.

Data Visualization:

It leverages Plotly Express (px) to create interactive charts, such as a bar chart for vulnerability distribution by risk level and a line chart for simulated risk trends over time.

Integration with Agent:

It calls the cyber_agent's generate_analytics and generate_industry_standard_report methods to fetch data directly for display.

Outcomes:

An interactive web dashboard that presents security assessment findings visually.

Clearer understanding of the types and severity of vulnerabilities.

Insights into tool usage and potential risk trends.
A professional interface for sharing assessment summaries.

How to Run the Streamlit Dashboard

Since Google Colab environments are designed for notebook execution, running a full-fledged Streamlit app directly within a single cell can be tricky. Typically, you'd save the Streamlit code to a .py file and then run it from your terminal.

Steps to run the Streamlit Dashboard (outside of a direct Colab cell):

Install necessary libraries:

Bash

!pip install streamlit plotly pandas
(Note: pandas is usually pre-installed in Colab)

Uncomment the Streamlit code: In the provided Python code, remove the # comments from the entire Streamlit dashboard block.

Save the code to a Python file: Copy the entire Python code (with the Streamlit block uncommented) into a new file named, for example, cyber_dashboard.py. You can do this in Colab by going to File > New file and pasting the content.

Run the Streamlit app: In a new Colab code cell, or from your terminal if you download the file, execute:

in shell/powershell : Bash

!streamlit run cyber_dashboard.py & npx localtunnel --port 8501

This command starts the Streamlit app and then uses localtunnel to create a public URL so you can access the dashboard from your browser. Colab often provides a public IP directly, but localtunnel is a reliable fallback.

After running, localtunnel will provide you with a public URL (e.g., https://xxxx-xxxx-xxxx.loca.lt) which you can open in your web browser to view the interactive dashboard.